<a href="https://colab.research.google.com/github/reedhodges/pytorch-loop-integrals/blob/main/pytorch_loop_integrals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using PyTorch to classify loop integrals by the type of their divergence

Before starting, run all the cells in the 'Preliminaries' section to make sure you've downloaded all the necessary files.

### Preliminaries

In [19]:
import torch
import requests
import zipfile
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"

path_to_data = Path("data/")

if path_to_data.is_dir():
    print(f"[INFO] Directory {path_to_data} already exists, skipping download.")
else:
    print(f"[INFO] Creating {path_to_data} directory...")
    path_to_data.mkdir(parents=True, exist_ok=True)
    with open("integrand_data.zip", "wb") as f:
        url = "https://github.com/reedhodges/pytorch-loop-integrals/raw/main/integrand_data.zip"
        response = requests.get(url)
        print(f"[INFO] Downloading zip from {url}...")
        f.write(response.content)

    with zipfile.ZipFile("integrand_data.zip", "r") as zip_ref:
        print(f"[INFO] Extracting zip...")
        zip_ref.extractall()

files_to_download = [
    {"path": "engine.py", "url": "https://raw.githubusercontent.com/reedhodges/pytorch-loop-integrals/main/engine.py"},
    {"path": "utils.py", "url": "https://raw.githubusercontent.com/reedhodges/pytorch-loop-integrals/main/utils.py"}
]

for file_info in files_to_download:
    file_path = file_info["path"]
    file_url = file_info["url"]
    if not Path(file_path).is_file():
        print(f"[INFO] Downloading {file_path}...")
        with open(file_path, "wb") as f:
            response = requests.get(file_url)
            f.write(response.content)
    else:
        print(f"[INFO] File {file_path} already exists, skipping download.")

print(f"[INFO] Done!")

[INFO] Directory data already exists, skipping download.
[INFO] File engine.py already exists, skipping download.
[INFO] File utils.py already exists, skipping download.
[INFO] Done!


### Set up data and model

In [41]:
image_path_list = list(path_to_data.glob("*/*/*.png"))

In [21]:
from utils import fix_error_with_weights_download
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights

# this is a hack to fix the weights download issue
fix_error_with_weights_download()

weights = EfficientNet_B0_Weights.DEFAULT
model = efficientnet_b0(weights=weights).to(device)

data_transform = weights.transforms()

for param in model.features.parameters():
    param.requires_grad = False

### Train

In [52]:
import os
from torch import nn, optim
from engine import create_data_loaders, train

BATCH_SIZE = 32
NUM_WORKERS = os.cpu_count()
NUM_EPOCHS = 3

train_loader, test_loader, class_names, class_dict = create_data_loaders(path_to_data, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, train_transform=data_transform, test_transform=data_transform)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280,
              out_features=len(class_names)).to(device)
)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

 
train(device, model, train_loader, test_loader, loss_fn, optimizer, epochs=NUM_EPOCHS)

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [02:32<05:04, 152.22s/it]


------------------------------

Epoch 1
Train Loss: 0.7174, Train Accuracy: 0.6750
Test Loss:  0.7240, Test Accuracy:  0.7200


 67%|██████▋   | 2/3 [05:03<02:31, 151.48s/it]


------------------------------

Epoch 2
Train Loss: 0.5855, Train Accuracy: 0.7400
Test Loss:  0.6200, Test Accuracy:  0.7450


100%|██████████| 3/3 [07:34<00:00, 151.50s/it]


------------------------------

Epoch 3
Train Loss: 0.5858, Train Accuracy: 0.7600
Test Loss:  0.5789, Test Accuracy:  0.7450
